# Code pour manipuler fichiers et faire le data set

Idée : 

1) télécharger les données du petit dataset https://www.dropbox.com/s/k88qukc20ljnbuo/PH2Dataset.rar?fbclid=IwAR2BEnfrRxPf-7Ri-eqdHkl4OjShyhY_42-R5Kx3vK4Z4SL0BmfOiCbMzF0

2) Il faut trier ces images en 3 dossiers : normaux, anormaux et mélanomes
    - Pour faire ça j'ai annoté le nom de chaque dossier suivant qu'il contenait une image d'un truc bénin, pas bénin, mélanome
    ex : dossier Im058 devient m_Im058 car il contenait un mélanome
    - comme ça tu peux facilement trier par préfixe et copier coller l'ensemble des dossier dans un nouveau dossier séparé
    
3) Maintenant que tu as les 3 dossiers de fait contenant chacun des dossiers, il faut extraire dans chaque sous dossier l'image et la stocker

4) Une fois ça de fait, c'est plus facile car il faut compléter le dataset avec les images qui viennent de là : https://www.isic-archive.com/?fbclid=IwAR1z5lIE-A_zGL1ewbxjQG6Q7AfRq4mgIsRO_afgb6jVFpVK2f0ittcHHWI#!/onlyHeaderTop/gallery

Tu peux par exemple télécharger 100 images de plus pour les bénins

pour la catégorie 2 "anormaux" tu peux rajouter des images en filtrant avec "unknown" (mais prends des trucs pertinents), "indeterminate"

pour la catégorie 3, en filtrant via malignant / melanomes


In [23]:
import numpy as np
import os
from matplotlib.pyplot import imread
from skimage.transform import resize
import shutil


Pour annoter facilement les dossiers j'ai créer 3 fichiers textes où sont stockés ligne par ligne le nom du dossier pour chaque cas genre melanomes.txt : IM058, IM069 etc

In [8]:
# choppe les noms des fichiers pour les 3 catégories
# possible qu'il te fasse chier avec le \abnormal
# si c'est le cas rename le car \a fait bugguer


melanomias = []

with open("data\PH2Dataset\melanome.txt", "r") as ifile:
    for line in ifile:
        melanomias.append(line.split("\n")[0])
            
            
benins = []

with open("data\PH2Dataset\benins.txt", "r") as ifile:
    for line in ifile:
        benins.append(line.split("\n")[0])
        
suspicious = []

with open("data\PH2Dataset\abnormal.txt", "r") as ifile:
    for line in ifile:
        suspicious.append(line.split("\n")[0])

In [31]:
# tu te places dans le dossier où il y a tous les dossiers des images
# tu itéres sur tous les dossiers et selon les cas tu modifies le préfixe

dirpath = "data\PH2Dataset\images"
for folder_name in os.listdir("data\PH2Dataset\images\."):
    if folder_name in melanomias:
        newname =  "m_" + folder_name 
        os.rename(os.path.join(dirpath, folder_name), os.path.join(dirpath, newname))
    elif folder_name in benins:
        newname =  "b_" + folder_name 
        os.rename(os.path.join(dirpath, folder_name), os.path.join(dirpath, newname))
    elif folder_name in suspicious:
        newname =  "s_" + folder_name 
        os.rename(os.path.join(dirpath, folder_name), os.path.join(dirpath, newname))
        
        

Là normalement si tout a bien marché tu as une longue liste de dossiers avec chacun un préfixe différent selon la caté, faut que tu crées **3 dossiers** appelés par ex : melanomas , suspicious, benins et tu copies colle dans chacun de ces dossiers les dossiers avec le préfixe qui va bien (tout ceux commencant par m_ dans le dossier melanoma etc)

Et là faut extraire les images maintenant

In [22]:
# tu te places dans le dossier melanoma et tu vas parcourir tous les dossiers et extraire les images dans un dossier appelé
# melanoma_img
# faut le faire pour chaque caté

dirpath = "data\melanoma"
target_path = "data\melanoma_img"
for folder_name in os.listdir(dirpath):
    input_path = os.path.join(dirpath, folder_name, folder_name.split("_")[1] + "_Dermoscopic_Image", folder_name.split("_")[1] + ".bmp" )
    shutil.copy2(input_path, target_path)


dirpath = "data\suspicious"
target_path = "data\suspicious_img"
for folder_name in os.listdir(dirpath):
    input_path = os.path.join(dirpath, folder_name, folder_name.split("_")[1] + "_Dermoscopic_Image", folder_name.split("_")[1] + ".bmp" )
    shutil.copy2(input_path, target_path)

    
dirpath = "data\benins"
target_path = "data\benins_img"
for folder_name in os.listdir(dirpath):
    input_path = os.path.join(dirpath, folder_name, folder_name.split("_")[1] + "_Dermoscopic_Image", folder_name.split("_")[1] + ".bmp" )
    shutil.copy2(input_path, target_path)

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'data\\melanoma'

Tu peux supprimer les listes de dossier et ne conserver que les 3 dossiers contenant les images des 3 catégories

Si tout s'est bien passé tu as **3 dossiers (bénins, suspicious et mélanomes)** avec des images

Maintenant il faut compléter ces images en telechargant d'autre images de la base ISIC cf ce que j'ai mis au début du notebook

Ensuite il faudra convertir et stocker ces images sous forme d'array d'arrays avec numpy

In [17]:
# prends en entrée le chemin du dossier contenant les images
# par exemple le dossier contenant toutes les images de mélanomes
# construit une liste avec le nom de chaque image par ex "IM058.bmp"
# ensuite on stock dans un np.array les images 1) rescalées en 400 * 400 2) en gray scale
# retourne un array d'array 

def from_images_to_npy_file(dirpath):
    img_name_liste = []
    for img_name in os.listdir(dirpath):
        img_name_liste.append(img_name)
        
    x = np.array([resize(imread(dirpath + fname).mean(axis=2), (400, 400), mode='reflect', anti_aliasing=True) for fname in img_name_liste])
    
    return x

In [20]:
# 1) convertir les images en array puis les stocker en array
# 2) .dump permet de stocker ça en fichier .npy comme dans mars cratere

array_melanomas = from_images_to_npy_file("data/moles_dataset/melanoma_img/")
array_melanomas.dump('data/moles_dataset/melanomias.npy')

array_abnormal = from_images_to_npy_file("data/moles_dataset/abnormal_img/")
array_abnormal.dump('data/moles_dataset/abnormal.npy')

array_normal = from_images_to_npy_file("data/moles_dataset/normal_img/")
array_normal.dump('data/moles_dataset/normal.npy')

**IMPORTANT**

Bon là normalement si tout s'est bien passé tu as 3 objets .npy qui contiennent les images 

Maintenant il faut créer 3 fchiers CSV qui contiennent les y

ça va dépendre du nombre d'images de chaque array mais par ex si array_melanomas contient 300 images donc shape = (300, ) où array_melanomas[0].shape = (300, 300) et bien il faut faire un fichier csv avec 300 fois "2" car ça sera de la classe 2 parmi (0, 1 ,2)

Ensuite il faut uploader ces 6 fichiers sur un cloud, je compte sur ta connexion pour ça ;)

In [4]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='12evFvHh2pUUkJEsBrgW4INj4oM39yTqG', dest_path='./data/benins.csv')

création du dataset :

récupérer les 3 fichiers contenant mélanomes - benins - suspicious

coupe chaque fichier 70% - 15% - 15% (M1, M2, M3 pour le fichier mélanome par ex)

stack 
- M1 - S1 - B1 <- train
- M2 - S2 - B2 <- test
- M3 - S3 - B3 <- validation

In [16]:
import numpy as np

melanomas  = np.load("./data/melanoma.npy")
benins     = np.load("./data/benins.npy")
suspicious = np.load("./data/suspicious.npy")

In [17]:
melanomas.shape

(208, 400, 400)

In [18]:
melanomas[0].shape

(400, 400)

In [34]:
M1, M2, M3 = np.split(melanomas, [158, 183])

print(M1.shape, M2.shape, M3.shape)

(158, 400, 400) (25, 400, 400) (25, 400, 400)


In [33]:
B1, B2, B3 = np.split(benins, [324, 394])

print(B1.shape, B2.shape, B3.shape)

(324, 400, 400) (70, 400, 400) (70, 400, 400)


In [35]:
suspicious.shape

(187, 400, 400)

In [39]:
S1, S2, S3 = np.split(suspicious, [137, 162])

print(S1.shape, S2.shape, S3.shape)

(137, 400, 400) (25, 400, 400) (25, 400, 400)


In [47]:
# id 0 -> 323 benin
# 324 -> 460 suspicious
# 461 -> end malin
data_train = np.vstack((B1, S1, M1))

# id 0 -> 69 benin
# 70 -> 94 suspicious
# 95 -> 119 malin
data_test  = np.vstack((B2, S2, M2))
data_validation  = np.vstack((B3, S3, M3))

In [53]:
data_train.shape

(619, 400, 400)

In [52]:
import pandas as pd

In [90]:
id_data_train = [i for i in range(0, 619)]
labels_data_train = [1 for i in range(0, 324)] + [2 for i in range(324, 461)] + [3 for i in range(461, 619)]

In [92]:
d = {'id': id_data_train, 'label': labels_data_train}
df = pd.DataFrame(data=d)


df.to_csv("./data/data_train_labels.csv", index=False)

In [106]:
id_data_test = [i for i in range(0, 120)]
labels_data_test = [1 for i in range(0, 70)] + [2 for i in range(70, 95)] + [3 for i in range(95, 120)]

In [109]:
d = {'id': id_data_test, 'label': labels_data_test}
df = pd.DataFrame(data=d)

df.to_csv("./data/data_test_labels.csv", index=False)

In [111]:
data_train.dump('data/data_train.npy')
data_test.dump('data/data_test.npy')
data_validation.dump('data/data_validation.npy')